# Bayesian optimization with `skopt`

(based on scikit-optimize documentation https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html)

In [0]:
# sklearn version fixed to avoid known skopt issue
!pip install scikit-optimize scikit-learn==0.20.3 

In [0]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


## Optimising a RandomForest classifier

In [0]:
from sklearn.datasets import load_digits
X, y = load_digits(10, True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75,
                                                    random_state=0)

dimensions for parameters  [list, shape=(n_dims,)]: List of search space dimensions. Each search dimension can be defined either as

- a (lower_bound, upper_bound) tuple (for Real or Integer dimensions),
- a (lower_bound, upper_bound, prior) tuple (for Real dimensions),
- as a list of categories (for Categorical dimensions), or
- an instance of a Dimension object (Real, Integer or Categorical).

In [0]:
param_dist = {
    "max_depth": (3, 10,),
    "max_features": (1, 11),
    "min_samples_split": <YOUR CODE>, # from 2 to 10
    "min_samples_leaf": <YOUR CODE>, # from 1 to 10
    "bootstrap": [True, False], # categorical valued parameter
    "criterion": <YOUR CODE> # either "gini" or "entropy"
   }

In [0]:
clf = RandomForestClassifier(n_estimators=20)
opt = BayesSearchCV(clf, param_dist, n_iter=10, return_train_score=True, cv=3)

In [0]:
opt.fit(X_train, y_train);

In [0]:
print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

In [0]:
# Utility function to report best scores
import pandas as pd

def report(results, n_top=3):
  res = pd.DataFrame(results)
  res = res.sort_values(by=['mean_test_score'], ascending=False, axis=0)
  res.reset_index(inplace = True, drop=True)
#   a = res[['mean_test_score', 'std_test_score']]

  for candidate in range(0, n_top):
    print("Model with rank: {0}".format(candidate))
    print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
          res['mean_test_score'][candidate],
          res['std_test_score'][candidate]))
    print("Parameters: {0}".format(res['params'][candidate]))
    print("")

In [0]:
report(opt.cv_results_)

## Neural Network
Optimise the neural net from the previous notebook via `BayesSearchCV` 


In [0]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
torch.manual_seed(0);

In [0]:
from sklearn.datasets import make_classification
X, y = make_classification(1000, 20, n_informative=10, n_classes=2, random_state=0)
X = X.astype(np.float32)

In [0]:
class ClassifierModule(nn.Module):
<CODE OF THE CLASSIFIER FROM NOTEBOOK i-1>

In [0]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    device='cuda',  # comment this to train with CPU
    optimizer__momentum=0.9,
    verbose=0
)

Define a space for parameter sampling in the form of dict, list of dict or list of tuple containing (dict, int). One of these cases: 

1. dictionary, where keys are parameter names (strings) and values are skopt.space.Dimension instances (Real, Integer or Categorical) or any other valid value that defines skopt dimension (see skopt.Optimizer docs). Represents search space over parameters of the provided estimator. 
2. list of dictionaries: a list of dictionaries, where every dictionary fits the description given in case 1 above. If a list of dictionary objects is given, then the search is performed sequentially for every parameter space with maximum number of evaluations set to self.n_iter. 
3. list of (dict, int > 0): an extension of case 2 above, where first element of every tuple is a dictionary representing some search subspace, similarly as in case 2, and second element is a number of iterations that will be spent optimizing over this subspace.

(see [skopt docs](https://scikit-optimize.github.io/#skopt.BayesSearchCV) for details)

In [0]:
params = {
    'lr': [0.05, 0.1],
    'module__num_units': [10, 20, 30], # range from 10 to 30
    'module__dropout': [0.1, 0.3], # range from 0.1 to 0.3
    'optimizer__nesterov': [False, True],
}

In [0]:
bs = BayesSearchCV(net, params, refit=False, cv=3, scoring='accuracy', 
                  verbose=0, n_jobs=1, n_iter=10, return_train_score=True)

In [0]:
bs.fit(X, y);

In [0]:
report(bs.cv_results_)

## Task
add `optimizer__momentum` to the space with [0.5, 1.5] range 